In [ ]:
from pathlib import Path
import numpy as np
import torch
from torch.utils.data import Dataset

class LandmarksDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = Path(root_dir)
        self.samples = []
        self.class_names = sorted([d.name for d in self.root_dir.iterdir() if d.is_dir()])
        self.class_to_idx = {c: i for i, c in enumerate(self.class_names)}

        for cls in self.class_names:
            for f in (self.root_dir / cls).glob("*.npy"):
                self.samples.append((f, self.class_to_idx[cls]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        x = np.load(path).astype("float32")
        x = torch.from_numpy(x)
        y = torch.tensor(label, dtype=torch.long)
        return x, y
# Example usage:
train_ds = LandmarksDataset("data/landmarks/train")
val_ds   = LandmarksDataset("data/landmarks/val")

In [ ]:
import torch
import torch.nn as nn

class LandmarksMLP(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(63, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.net(x)


In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

# --------------------
# CONFIG
# --------------------
BATCH_SIZE = 64
EPOCHS = 30
LR = 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# --------------------
# DATA
# --------------------
train_ds = LandmarksDataset("data/landmarks/train")
val_ds   = LandmarksDataset("data/landmarks/val")

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE)

NUM_CLASSES = len(train_ds.class_names)

# --------------------
# MODEL
# --------------------
model = LandmarksMLP(NUM_CLASSES).to(DEVICE)
optimizer = Adam(model.parameters(), lr=LR)
criterion = CrossEntropyLoss()

# --------------------
# TRAIN LOOP
# --------------------
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for x, y in train_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)

        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # VALIDATION
    model.eval()
    correct = total = 0

    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            preds = model(x).argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

    acc = correct / total
    print(f"Epoch {epoch+1}/{EPOCHS} | loss={total_loss:.3f} | val_acc={acc:.3f}")

# SAVE
torch.save(model.state_dict(), "checkpoints/landmarks_mlp.pt")
